#注意
- stdio 输入输出记得不要用print
- 工具使用格式参考 https://modelcontextprotocol.io/specification/draft/server/tools#tool-names
- mcp server primatives：
    1. resources：data
    2. tools： funtion call
    3. prompts：templates
- mcp服务器的搭建总共分为几步：
    1. 安装依赖环境
    2. 设计暴露api接口格式和请求格式
    3. 解析验证用户端的数据
    2. 写 helper 函数 function call， 设计封装
    3. 启动服务器

In [ ]:
##环境设计
!curl -LsSf https://astral.sh/uv/install.sh | sh
# Create a new directory for our project
!uv init 服务器名字
!cd 服务器名字

# Create virtual environment and activate it
!uv venv
!source .venv/bin/activate

# Install dependencies
!uv add "mcp[cli]" httpx

# Create our server file
!touch 服务器名字.py

In [ ]:
##搭建服务器功能
from typing import Any
import httpx
# 引入FastMCP
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("服务器名字")

# Constants
#API_BASE = "https://api.服务器名字.gov"
#USER_AGENT = "服务器名字-app/1.0"

In [ ]:
##加入helper funtion来设计api请求和回复数据格式
async def make_nws_request(url: str) -> dict[str, Any] | None:
    """Make a request to the NWS API with proper error handling."""
    headers = {
        "User-Agent": USER_AGENT,
        "Accept": "application/geo+json"
    }
    async with httpx.AsyncClient() as client:
        try:
            response = await client.get(url, headers=headers, timeout=30.0)
            response.raise_for_status()
            return response.json()
        except Exception:
            return None

def format_alert(feature: dict) -> str:
    """Format an alert feature into a readable string."""
    props = feature["properties"]
    return f"""
Event: {props.get('event', 'Unknown')}
Area: {props.get('areaDesc', 'Unknown')}
Severity: {props.get('severity', 'Unknown')}
Description: {props.get('description', 'No description available')}
Instructions: {props.get('instruction', 'No specific instructions provided')}
"""

In [ ]:
## 加入工具执行，负责每个function call具体功能
##function call 示例

@mcp.tool()
async def get_alerts(state: str) -> str:
    """Get weather alerts for a US state.

    Args:
        state: Two-letter US state code (e.g. CA, NY)
    """
    url = f"{NWS_API_BASE}/alerts/active/area/{state}"
    data = await make_nws_request(url)

    if not data or "features" not in data:
        return "Unable to fetch alerts or no alerts found."

    if not data["features"]:
        return "No active alerts for this state."

    alerts = [format_alert(feature) for feature in data["features"]]
    return "\n---\n".join(alerts)

@mcp.tool()
async def get_forecast(latitude: float, longitude: float) -> str:
    """Get weather forecast for a location.

    Args:
        latitude: Latitude of the location
        longitude: Longitude of the location
    """
    # First get the forecast grid endpoint
    points_url = f"{NWS_API_BASE}/points/{latitude},{longitude}"
    points_data = await make_nws_request(points_url)

    if not points_data:
        return "Unable to fetch forecast data for this location."

    # Get the forecast URL from the points response
    forecast_url = points_data["properties"]["forecast"]
    forecast_data = await make_nws_request(forecast_url)

    if not forecast_data:
        return "Unable to fetch detailed forecast."

    # Format the periods into a readable forecast
    periods = forecast_data["properties"]["periods"]
    forecasts = []
    for period in periods[:5]:  # Only show next 5 periods
        forecast = f"""
{period['name']}:
Temperature: {period['temperature']}°{period['temperatureUnit']}
Wind: {period['windSpeed']} {period['windDirection']}
Forecast: {period['detailedForecast']}
"""
        forecasts.append(forecast)

    return "\n---\n".join(forecasts)

### 运行服务器
ai application 的连接不在服务器端完成，他被认为用户端的输入。这里的服务器可以作为连接入claude ai的插件进行功能扩展。\
运行服务器：
```
uv run 服务器名字.py
```
- 为接入claude ai 或其他大语言模型，需要打开config.json文件加入插件、
看起来像这样：

```json
{
  "mcpServers": {
    "服务器名字(mcp名称)": {
      "command": "path/to/uv",
      "args": [
        "--directory",
        "/ABSOLUTE/PATH/TO/PARENT/FOLDER/服务器名字",
        "run",
        "服务器名字.py"
      ]
    }
  }
}
```
保存后，重新运行服务器即可。


## 改进思路：
1. 增加http 异步处理和开发
2. 错误处理和重试机制
3. 扩展功能，增加functio
*5. 容器化 kubernetes